# 快速入门 GPT-4 Vison

从历史上看，语言模型系统仅接受**文本**作为输入。但是单一的输入形式，限制了大模型的应用落地范围。

随着技术发展，OpenAI 开发的 GPT-4 Turbo with Vision（简称 GPT-4V）允许模型接收**图像**作为输入，并回答关于它们的问题。

📢注意，目前在 Assistants API 中使用 GPT-4 时还不支持图像输入。

## 使用 GPT-4V 识别线上图像（URL）

![image_sample](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg)

In [25]:
from openai import OpenAI
import os

client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'),base_url='https://api.xiaoai.plus/v1')

response = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "介绍下这幅图?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='这幅图展示了一条穿越湿地或草地的木制小径。图片中，木板路笔直延伸，两旁是郁郁葱葱的草地和灌木，远处可见树木和清晰的蓝天，其中夹杂着一些细小的白云。整个场景给人一种宁静和平和的感觉，是自然风光摄影中常见的主题，展现了户外探索和自然之美的元素。这样的路径可能是为了方便游客或者自然爱好者更好地探访和欣赏这片自然环境而建造的。', role='assistant', function_call=None, tool_calls=None))


In [26]:
response.choices[0].message.content

'这幅图展示了一条穿越湿地或草地的木制小径。图片中，木板路笔直延伸，两旁是郁郁葱葱的草地和灌木，远处可见树木和清晰的蓝天，其中夹杂着一些细小的白云。整个场景给人一种宁静和平和的感觉，是自然风光摄影中常见的主题，展现了户外探索和自然之美的元素。这样的路径可能是为了方便游客或者自然爱好者更好地探访和欣赏这片自然环境而建造的。'

### 封装成一个函数 query_image_description

In [27]:
def query_image_description(url, prompt="介绍下这幅图?"):
    client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'),base_url='https://api.xiaoai.plus/v1')  # 初始化 OpenAI 客户端
    
    # 发送请求给 OpenAI 的聊天模型
    response = client.chat.completions.create(
        model="gpt-4-turbo",  # 指定使用的模型
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": url}},
                ],
            }
        ],
        max_tokens=300,
    )
    
    # 返回模型的响应
    return response.choices[0].message.content


### 调用函数测试

![meme_0](https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg)

In [28]:
image_url = "https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg"
content = query_image_description(image_url)
print(content)

这幅图是一张幽默搞笑的图片，展示了两只不同形象的柴犬。左边的柴犬被P图成一个巨大的、肌肉发达的人形身体，看起来很像超级英雄或健身达人，配文说这是“16岁的我，工作后的我”。右边的柴犬则保持了正常的狗的形态，呈现出一种憨态可掬、略显懒散的样子，显得格外可爱和无奈。

这幅图片通过夸张和对比，幽默地表达了很多人在青少年时期满怀志向、精力充沛（代表左边的肌肉柴犬），而工作多年后可能变得沉闷、疲惫或失去了最初的激情和活力（代表右边的普通柴犬）。这种对比反映了社会现实和个人生活的一种普遍感


### 使用 GPT-4V 识别本地图像文件（Base64编码）


In [29]:
from openai import OpenAI
import base64
import requests
import json

client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'),base_url='https://api.xiaoai.plus/v1')  # 初始化 OpenAI 客户端

def query_base64_image_description(image_path, prompt="解释下图里的内容？", max_tokens=1000):

    # 实现 Base64 编码
    def encode_image(path):
        with open(path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 获取图像的 Base64 编码字符串
    base64_image = encode_image(image_path)

    # 构造请求的 HTTP Header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }

    # 构造请求的负载
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": max_tokens,
    }

    # 发送 HTTP 请求
    response = requests.post("https://api.xiaoai.plus/v1/chat/completions", headers=headers, json=payload)

    # 检查响应并提取所需的 content 字段
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        return content
    else:
        return f"Error: {response.status_code}, {response.text}"

#### 使用 Assistants API生成的 GDP 40年对比曲线图

![gdp_data](./images/gdp_1980_2020.jpg)

In [30]:
content = query_base64_image_description("./images/gdp_1980_2020.jpg")
print(content)

这张图表展示了1980年至2020年间美国、中国、日本和德国的GDP（国内生产总值）的对比，单位为万亿美元（Trillion USD）。以下是图表的一些主要观察点：

1. **美国（蓝色曲线）**：
   - 从1980年到2020年，美国的GDP持续增长。
   - 增长趋势大致平稳，但在2008年左右的全球金融危机期间增长速度有所放缓。
   - 到2020年，美国的GDP超过20万亿美元。

2. **中国（红色曲线）**：
   - 从1980年到2000年，中国的GDP增长相对较慢。
   - 从2000年开始，中国的GDP迅速增长，特别是2008年后的增速显著加快。
   - 到2020年，中国的GDP接近15万亿美元，明显超过日本和德国，成为全球第二大经济体。

3. **日本（紫色曲线）**：
   - 1980年至1995年，日本的GDP持续增长，并在1995年左右达到峰值。
   - 之后，日本的GDP经历了一个相对平稳或略有下降的阶段。
   - 到2020年，日本的GDP大约保持在5万亿美元左右。

4. **德国（绿色曲线）**：
   - 1980年到2020年间，德国的GDP呈现稳定增长的趋势。
   - 尽管增长幅度较小，但整体趋势向上。
   - 到2020年，德国的GDP略超过4万亿美元。

总体而言，这张图表展示了美国作为全球最大经济体的稳定增长，同时也突出了中国经济的快速崛起。日本和德国的经济增长较为稳定，但与中美两国相比增速较慢。


#### 使用 GPT-4V 识别手写体笔记

![](./images/handwriting_0.jpg)

In [31]:
content = query_base64_image_description("./images/handwriting_0.jpg")
print(content)

这张图片展示的是一本笔记本上记载的一些有关自然语言处理（NLP）中模型调整（tuning）技术的笔记。笔记主要分为几个部分，涉及到不同的调整技术和方法，主要包括：

1. **Prompt Tuning**：提到了使用嵌入（embedding）的输入 "X" (X1, X2, ..., Xn)，通过变换矩阵 "W" 转换为输出 "Y"。这种方法通常用于少数样本中的模型微调，特别是适合小模型。

2. **Prefix Tuning**：在这部分，笔记指出增加了前缀 "Wp" 到权重中，表示这是一种通过调整输入序列前端的方式来预训练Transformer模型的方法。

3. **LoRA**（Low-Rank Adaptation）：这部分描述了通过调整权重矩阵 "W" 并添加一个低秩更新矩阵 "ΔW"，可以改变模型的行为。这里用了矩阵分解的方式来解释 "ΔW" = AB，其中 A 和 B 是更小的矩阵。

4. 笔记底部提到了一些性能和存储需求的数据，如 "LAMA-65TB" 和 "QLoRA-48GB"，可能表示使用这些技术的不同配置或方案所需的资源。

总之，这些笔记涵盖了NLP中一些先进的模型微调技术，特别是关注于减少模型调整对大量数据依赖和显著减少计算资源需求的方法。


#### 在 Jupyter 标准输出中渲染 Markdown 格式内容

In [32]:
from IPython.display import display, Markdown

# 使用 display 和 Markdown 函数显示 Markdown 内容
display(Markdown(content))

这张图片展示的是一本笔记本上记载的一些有关自然语言处理（NLP）中模型调整（tuning）技术的笔记。笔记主要分为几个部分，涉及到不同的调整技术和方法，主要包括：

1. **Prompt Tuning**：提到了使用嵌入（embedding）的输入 "X" (X1, X2, ..., Xn)，通过变换矩阵 "W" 转换为输出 "Y"。这种方法通常用于少数样本中的模型微调，特别是适合小模型。

2. **Prefix Tuning**：在这部分，笔记指出增加了前缀 "Wp" 到权重中，表示这是一种通过调整输入序列前端的方式来预训练Transformer模型的方法。

3. **LoRA**（Low-Rank Adaptation）：这部分描述了通过调整权重矩阵 "W" 并添加一个低秩更新矩阵 "ΔW"，可以改变模型的行为。这里用了矩阵分解的方式来解释 "ΔW" = AB，其中 A 和 B 是更小的矩阵。

4. 笔记底部提到了一些性能和存储需求的数据，如 "LAMA-65TB" 和 "QLoRA-48GB"，可能表示使用这些技术的不同配置或方案所需的资源。

总之，这些笔记涵盖了NLP中一些先进的模型微调技术，特别是关注于减少模型调整对大量数据依赖和显著减少计算资源需求的方法。

![](./images/handwriting_1.jpg)

In [33]:
content = query_base64_image_description("./images/handwriting_1.jpg")
display(Markdown(content))

这张图片显示的是一本笔记本上记录的笔记内容，主要涉及自然语言处理（NLP）和机器学习中的一些技术和方法。内容主要围绕“Transformer”模型及其各种调整（Tuning）方法，包括Prompt Tuning和Adapter等技术。

在左侧页面顶部，提到“一代Transformers”和“检索”，可能是在概括Transformer模型在信息检索方面的应用或进展。下面列表化了PEFT、SOTA、PFT Methods等术语，这些可能是指与预训练或微调模型相关的不同方法或标准。

在右侧页面，“multi-modality Instruction FT. LLMs”可能指的是多模态指令微调的大型语言模型（Large Language Models，LLMs）。"LILA、LOLA、GLORA、Adolora" 可能是指具体的方法或模型名称。同时，还提到了“PETC”、“prefix-tuning & Adapters” 等术语，这些技术通常用于调整预训练模型以更好地适应特定任务或数据集。

整体上，这些笔记反映了作者在理解并记录当前最新的自然语言处理技术，特别是在模型微调和适应技术方面的学习或研究进展。

## Homework: 


### #1

使用 GPT-4V 识别带有手写体文字的本地图像文件，分享结果。

### #2

整合 `query_base64_image_description` 函数和 Markdown 格式渲染方法，使得输出结果更易阅读。

![](./images/gpt-4v.jpg)

In [23]:
content = query_base64_image_description(image_path="./images/gpt-4v.jpg", prompt="请解释下图里的内容，并且当图中的X是2，y是1的时候，结果是多少？")
display(Markdown(content))

上图展示了一个数学式的推导过程。解释如下：

1. 原始公式是： \((x + y + 1) \times (x + y - 1)\)
2. 这通过差平方公式可以简化为: \((x + y)^2 - 1^2\)
3. 进一步展开为: \(x^2 + 2xy + y^2 - 1\)

现在，如果 \(x = 2\) 和 \(y = 1\)：
- 将这些值代入公式 \(x^2 + 2xy + y^2 - 1\) 中。

计算过程如下：
1. \(x^2 = 2^2 = 4\)
2. \(2xy = 2 \times 2 \times 1 = 4\)
3. \(y^2 = 1^2 = 1\)

所以，\(x^2 + 2xy + y^2 - 1 = 4 + 4 + 1 - 1 = 8\)

因此，当 \(x = 2\) 和 \(y = 1\) 的时候，结果是 8。